In [1]:
import pandas as pd
import sys, os, glob
import gezi, melt
import traceback
from scipy.stats import rankdata
import numpy as np
from sklearn.preprocessing import minmax_scale
import sys
sys.path.append('../src')
from evaluate import evaluate
import sys
from config import *
import melt
melt.init_flags()
FLAGS = melt.get_flags()
from gezi import tqdm

In [2]:
dv = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation.csv')

In [3]:
dt = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [4]:
dve = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/translate/validation-en.csv')

In [5]:
dte = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/translate/test-en.csv')

In [6]:
len(dte)

63812

In [7]:
pd.set_option('display.max_colwidth', -1)

In [8]:
dve.head(2)

,id,comment_text,translated,lang,toxic
0,0,"Este usuario ni siquiera llega al rango de hereje . Por lo tanto debería ser quemado en la barbacoa para purificar su alma y nuestro aparato digestivo mediante su ingestión. Skipe linkin 22px Honor, valor, leltad. 17:48 13 mar 2008 (UTC)","This user does not even make it to the rank of heretic . It should therefore be burned in the barbecue to purify his soul and to our digestive system through ingestion. Skipe linkin 22px Honor, courage, leltad. 17:48 13 mar 2008 (UTC)",es,0
1,1,"Il testo di questa voce pare esser scopiazzato direttamente da qui. Immagino possano esserci problemi di copyright, nel fare cio .",The text of this entry appears to be like I directly from here. I guess there may be copyright issues in doing this .,it,0


In [9]:
!ls ../working/exps/xlm/

cv-toxic-mix
cv-toxic-mix-bylang
cv-toxic-mix-mdrop
cv-toxic-mix-mdrop-bylang
cv-toxic-multi-en-finetune-unint-large
cv-toxic-multi-en-finetune-unint-large-bylang
cv-toxic-multi-en-finetune-unint-large-mdrop
cv-toxic-multi-en-finetune-unint-large-mdrop-bylang
cv-toxic-multi-en-finetune-unint-large-mdrop-old
cv-toxic-multi-en-finetune-unint-large-mdrop-old-bylang
cv-toxic-multi-en-finetune-unint-large-old
cv-toxic-multi-en-finetune-unint-large-old-bylang
cv-toxic-unintended
cv-toxic-unintended2
cv-toxic-unintended-bylang
debug
debug0
debug2
test
toxic-from-mix
toxic-mix
toxic-mix-unintended
toxic-unintended


In [10]:
!ls ../working/exps/xlm-pair/

common			    cv-toxic-mix-pair-bylang
cv-debug10		    cv-toxic-multi-en-finetune-unint-large
cv-debug10-bylang	    cv-toxic-multi-en-finetune-unint-large-bylang
cv-debug2		    cv-toxic-multi-en-finetune-unint-large-mdrop
cv-debug2-bylang	    cv-toxic-multi-en-finetune-unint-large-mdrop-bylang
cv-debug5		    debug
cv-debug5-bylang	    debug3
cv-debug8		    debug4
cv-debug8-bylang	    debug6
cv-debug9		    debug6-bylang
cv-debug9-bylang	    test
cv-toxic-mix-pair	    toxic-mix-unintended
cv-toxic-mix-pair2	    toxic-mix-unintended2
cv-toxic-mix-pair2-baylang  trans


In [11]:
def read_valid(dir, merge=True):
  if os.path.exists(f'{dir}/cv/valid.csv'):
    dfs = []
    df = pd.read_csv(f'{dir}/cv/valid.csv')
  else:
    dfs = [pd.read_csv(x) for x in glob.glob(f'{dir}/infos/valid_*.csv')]
    df = pd.concat(dfs)
  if merge:
    df = pd.merge(df, dve, on='id')[['id', 'label', 'pred', 'translated', 'lang', 'comment_text']]
  assert len(df) == len(dv), f'{len(df)} {len(dfs)}'
  df = df.sort_values('id')
  return df

def eval(df, name):
  if 'label' not in df.columns:
    df.label = df.toxic
  result = evaluate(df.label, df.pred, df)
  result['name'] = name
  return result
  
def show_metrics(dir):
  name = '/'.join(dir.split('/')[-2:])
  try:
    df = read_valid(dir)
    df['name'] = name
    results = eval(df, name)
    gezi.pprint(results)
  except Exception:
    traceback.print_exc()
    pass
  try:
    df2 = read_valid(f'{dir}-bylang')
    df2[name] = name
    eval(df2, name + '-bylang')
    results = eval(df2, name)
    gezi.pprint(results)
  except Exception:
    pass
  
def read_valids(dir):
  name = '/'.join(dir.split('/')[-2:])
  df = read_valid(dir)
  df['name'] = name
  df2 = read_valid(f'{dir}-bylang')
  df2[name] = name
  return df, df2

def read_test(dir):
  if os.path.exists(f'{dir}/cv/submission.csv'):
    dfs = []
    df = pd.read_csv(f'{dir}/cv/submission.csv')
  else:
    dfs = [pd.read_csv(x) for x in glob.glob(f'{dir}/infos/test_*.csv')]
    assert len(dfs) == 5
    assert len(dfs) > 0, dir
    df = pd.concat(dfs).groupby('id', as_index=False).mean()
  
  assert len(df) == len(dt), f'{len(df)} {len(dfs)}, {dir}'
  df = pd.merge(df, dte, on='id')[['id', 'toxic', 'translated', 'lang', 'content']]
  df = df.sort_values('id')
  return df

def read_tests(dir):
  df = read_test(dir)
  try:
    df2 = read_test(f'{dir}-bylang')
  except Exception:
    df2 = None
  return df, df2

In [12]:
show_metrics('../working/exps/v2/cv-toxic-multi-en-finetune-unint-large')


 +----------+-----------+----------+----------+----------+-------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                      |
|----------+-----------+----------+----------+----------+-------------------------------------------|
| 0.199247 |  0.947923 | 0.930354 | 0.914224 | 0.983888 | v2/cv-toxic-multi-en-finetune-unint-large |
+----------+-----------+----------+----------+----------+-------------------------------------------+


In [13]:
show_metrics('../working/exps/xlm/cv-toxic-multi-en-finetune-unint-large')


 +----------+-----------+----------+----------+----------+--------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                       |
|----------+-----------+----------+----------+----------+--------------------------------------------|
| 0.283819 |  0.945901 | 0.922055 |  0.91382 | 0.984441 | xlm/cv-toxic-multi-en-finetune-unint-large |
+----------+-----------+----------+----------+----------+--------------------------------------------+

 +----------+-----------+----------+----------+----------+--------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                       |
|----------+-----------+----------+----------+----------+--------------------------------------------|
| 0.337029 |  0.925148 | 0.919403 | 0.898642 | 0.980316 | xlm/cv-toxic-multi-en-finetune-unint-large |
+----------+-----------+----------+----------+----------+------------

In [14]:
show_metrics('../working/exps/xlm/cv-toxic-multi-en-finetune-unint-large-mdrop')


 +----------+-----------+----------+----------+----------+--------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                             |
|----------+-----------+----------+----------+----------+--------------------------------------------------|
| 0.319622 |  0.946249 | 0.924262 | 0.914941 | 0.982102 | xlm/cv-toxic-multi-en-finetune-unint-large-mdrop |
+----------+-----------+----------+----------+----------+--------------------------------------------------+

 +----------+-----------+----------+----------+----------+--------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                             |
|----------+-----------+----------+----------+----------+--------------------------------------------------|
| 0.370718 |  0.924685 | 0.920327 | 0.892403 | 0.978834 | xlm/cv-toxic-multi-en-finetune-unint-large-mdrop |
+----------+---

In [15]:
!ls ../working/exps/xlm/cv-toxic-multi-en-finetune-unint-large-mdrop-bylang/cv

model_0.h5  model_1.h5	model_2.h5  tmp.h5  valid.csv


In [16]:
show_metrics('../working/exps/xlm/cv-toxic-multi-en-finetune-unint-large-old')


 +----------+-----------+----------+----------+----------+------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                           |
|----------+-----------+----------+----------+----------+------------------------------------------------|
| 0.278315 |  0.947813 | 0.925435 | 0.917774 | 0.983008 | xlm/cv-toxic-multi-en-finetune-unint-large-old |
+----------+-----------+----------+----------+----------+------------------------------------------------+

 +----------+-----------+----------+----------+----------+------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                           |
|----------+-----------+----------+----------+----------+------------------------------------------------|
| 0.348721 |   0.93071 | 0.923939 | 0.902529 | 0.978039 | xlm/cv-toxic-multi-en-finetune-unint-large-old |
+----------+-----------+---------

In [17]:
show_metrics('../working/exps/xlm/cv-toxic-multi-en-finetune-unint-large-mdrop-old')


 +----------+-----------+----------+----------+----------+------------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                                 |
|----------+-----------+----------+----------+----------+------------------------------------------------------|
| 0.278363 |  0.947218 | 0.924197 |  0.91568 | 0.983216 | xlm/cv-toxic-multi-en-finetune-unint-large-mdrop-old |
+----------+-----------+----------+----------+----------+------------------------------------------------------+

 +----------+-----------+----------+----------+----------+------------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                                 |
|----------+-----------+----------+----------+----------+------------------------------------------------------|
| 0.326821 |  0.932707 | 0.921836 | 0.907648 | 0.976035 | xlm/cv-toxic-multi-en-finetune-uni

In [18]:
show_metrics('../working/exps/xlm/cv-toxic-mix')


 +----------+-----------+----------+----------+----------+------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name             |
|----------+-----------+----------+----------+----------+------------------|
| 0.203307 |  0.948511 | 0.921323 |  0.91886 | 0.985599 | xlm/cv-toxic-mix |
+----------+-----------+----------+----------+----------+------------------+

 +----------+-----------+----------+----------+----------+------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name             |
|----------+-----------+----------+----------+----------+------------------|
| 0.232598 |  0.933887 | 0.921439 |  0.90482 | 0.979651 | xlm/cv-toxic-mix |
+----------+-----------+----------+----------+----------+------------------+


In [19]:
show_metrics('../working/exps/xlm/cv-toxic-mix-mdrop')


 +----------+-----------+----------+----------+----------+------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                   |
|----------+-----------+----------+----------+----------+------------------------|
| 0.218142 |  0.947161 | 0.918152 | 0.918964 | 0.984805 | xlm/cv-toxic-mix-mdrop |
+----------+-----------+----------+----------+----------+------------------------+

 +----------+-----------+----------+----------+----------+------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                   |
|----------+-----------+----------+----------+----------+------------------------|
| 0.238537 |  0.936728 | 0.920485 | 0.904579 | 0.981473 | xlm/cv-toxic-mix-mdrop |
+----------+-----------+----------+----------+----------+------------------------+


In [20]:
show_metrics('../working/exps/xlm-pair/cv-debug2')


 +----------+-----------+----------+----------+----------+--------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|----------+-----------+----------+----------+----------+--------------------|
| 0.209074 |  0.950267 | 0.927335 | 0.918267 | 0.985589 | xlm-pair/cv-debug2 |
+----------+-----------+----------+----------+----------+--------------------+

 +---------+-----------+----------+----------+----------+--------------------+
|    loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|---------+-----------+----------+----------+----------+--------------------|
| 0.25412 |  0.931811 |  0.92089 | 0.895111 | 0.982178 | xlm-pair/cv-debug2 |
+---------+-----------+----------+----------+----------+--------------------+


In [21]:
show_metrics('../working/exps/xlm-pair/cv-debug5')


 +----------+-----------+----------+----------+----------+--------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|----------+-----------+----------+----------+----------+--------------------|
| 0.219925 |  0.949793 | 0.926174 | 0.913599 |  0.98714 | xlm-pair/cv-debug5 |
+----------+-----------+----------+----------+----------+--------------------+

 +----------+-----------+----------+----------+----------+--------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|----------+-----------+----------+----------+----------+--------------------|
| 0.276038 |  0.933002 | 0.920997 | 0.895653 | 0.983039 | xlm-pair/cv-debug5 |
+----------+-----------+----------+----------+----------+--------------------+


In [22]:
show_metrics('../working/exps/xlm-pair/cv-debug8')


 +----------+-----------+----------+----------+----------+--------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|----------+-----------+----------+----------+----------+--------------------|
| 0.220467 |   0.94815 |  0.92457 | 0.913488 | 0.985338 | xlm-pair/cv-debug8 |
+----------+-----------+----------+----------+----------+--------------------+

 +----------+-----------+----------+----------+----------+--------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name               |
|----------+-----------+----------+----------+----------+--------------------|
| 0.281877 |  0.933433 |  0.92242 | 0.900973 | 0.984181 | xlm-pair/cv-debug8 |
+----------+-----------+----------+----------+----------+--------------------+


In [23]:
show_metrics('../working/exps/xlm-pair/cv-toxic-mix-pair')


 +----------+-----------+----------+----------+----------+----------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                       |
|----------+-----------+----------+----------+----------+----------------------------|
| 0.232719 |  0.942092 | 0.924325 | 0.897427 | 0.984806 | xlm-pair/cv-toxic-mix-pair |
+----------+-----------+----------+----------+----------+----------------------------+

 +---------+-----------+----------+----------+----------+----------------------------+
|    loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                       |
|---------+-----------+----------+----------+----------+----------------------------|
| 0.26295 |  0.929914 | 0.922246 | 0.891644 | 0.983692 | xlm-pair/cv-toxic-mix-pair |
+---------+-----------+----------+----------+----------+----------------------------+


In [24]:
show_metrics('../working/exps/xlm-pair/cv-toxic-multi-en-finetune-unint-large')


 +----------+-----------+----------+----------+----------+-------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                            |
|----------+-----------+----------+----------+----------+-------------------------------------------------|
| 0.329557 |  0.945107 | 0.914951 | 0.913141 | 0.986753 | xlm-pair/cv-toxic-multi-en-finetune-unint-large |
+----------+-----------+----------+----------+----------+-------------------------------------------------+

 +----------+-----------+----------+----------+----------+-------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                            |
|----------+-----------+----------+----------+----------+-------------------------------------------------|
| 0.390636 |  0.923868 |  0.91448 | 0.877189 | 0.980809 | xlm-pair/cv-toxic-multi-en-finetune-unint-large |
+----------+-----------+

In [25]:
show_metrics('../working/exps/xlm-pair/cv-toxic-multi-en-finetune-unint-large-mdrop')


 +----------+-----------+----------+----------+----------+-------------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                                  |
|----------+-----------+----------+----------+----------+-------------------------------------------------------|
| 0.205271 |  0.949219 | 0.929535 | 0.917462 | 0.983653 | xlm-pair/cv-toxic-multi-en-finetune-unint-large-mdrop |
+----------+-----------+----------+----------+----------+-------------------------------------------------------+

 +----------+-----------+----------+----------+----------+-------------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                                  |
|----------+-----------+----------+----------+----------+-------------------------------------------------------|
| 0.243609 |  0.933947 | 0.928675 | 0.899381 | 0.980068 | xlm-pair/cv-toxic-multi-en

In [26]:
root = '../working/exps/'
models = [
  'xlm/cv-toxic-multi-en-finetune-unint-large-old',
  'xlm/cv-toxic-multi-en-finetune-unint-large-mdrop-old',
  'xlm/cv-toxic-mix',
  'xlm/cv-toxic-mix-mdrop',
  'xlm-pair/cv-debug8',
  'xlm-pair/cv-debug5',
]

In [27]:
dfs, dfs2 = [], []
for model in models:
  df, df2 = read_valids(f'{root}/{model}')
  dfs += [df]
  dfs2 += [df2]

In [28]:
df = pd.concat(dfs).groupby('id', as_index=False).mean()
df = pd.merge(df, dve, on='id')
gezi.pprint(evaluate(df.toxic, df.pred, df))


 +----------+-----------+----------+----------+----------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr |
|----------+-----------+----------+----------+----------|
| 0.194386 |  0.954716 | 0.931297 |  0.92633 | 0.987796 |
+----------+-----------+----------+----------+----------+


In [29]:
df2 = pd.concat(dfs2).groupby('id', as_index=False).mean()
df2 = pd.merge(df2, dve, on='id')[['id', 'toxic', 'pred', 'lang']]
gezi.pprint(evaluate(df2.toxic, df2.pred, df2))
df2


 +---------+-----------+----------+----------+----------+
|    loss |   auc/all |   auc/es |   auc/it |   auc/tr |
|---------+-----------+----------+----------+----------|
| 0.23037 |  0.939336 | 0.928304 | 0.908048 | 0.983612 |
+---------+-----------+----------+----------+----------+


,id,toxic,pred,lang,y_true
0,0,0,0.461195,es,0
1,1,0,0.000281,it,0
2,2,1,0.685151,es,1
3,3,0,0.000427,tr,0
4,4,0,0.000685,tr,0
...,...,...,...,...,...
7995,7995,0,0.000241,it,0
7996,7996,1,0.988563,es,1
7997,7997,1,0.659842,tr,1
7998,7998,0,0.000615,es,0


In [30]:
def blend_byrank(weights):
  ranked = rankdata(weights)
  sum_rank = np.sum(ranked)
  for j in range(len(weights)):
    weights[j] = ranked[j] / sum_rank
    
def blend_byweight(weights, norm_facotr=0.1, scale=False):
  if scale:
    weights = minmax_scale(weights)
  min_ws = np.min(weights)
  max_ws = np.max(weights)
  gap = max_ws - min_ws
  if gap > 0:
   for i in range(len(weights)):
     weights[i] = ((weights[i] - min_ws) / gap) + norm_facotr

In [31]:
weights_dict = {}
langs = ['es', 'it', 'tr']
weights_dict['all'] = []
for lang in langs:
  weights_dict[lang] = []
dfs, dfs2, metrics = [], [], []
for model in models:
  df, df2 = read_valids(f'{root}/{model}')
  dfs += [df]
  dfs2 += [df2]
  metrics += [(eval(df, model), eval(df2, model))]
  print(model)
  gezi.pprint(metrics[-1][0])
  gezi.pprint(metrics[-1][1])
  weights_dict['all'] += [metrics[-1][1]['auc/all']]
  for lang in langs:
    weights_dict[lang] += [metrics[-1][0][f'auc/{lang}']]
weights_dict

xlm/cv-toxic-multi-en-finetune-unint-large-old

 +----------+-----------+----------+----------+----------+------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                           |
|----------+-----------+----------+----------+----------+------------------------------------------------|
| 0.278315 |  0.947813 | 0.925435 | 0.917774 | 0.983008 | xlm/cv-toxic-multi-en-finetune-unint-large-old |
+----------+-----------+----------+----------+----------+------------------------------------------------+

 +----------+-----------+----------+----------+----------+------------------------------------------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name                                           |
|----------+-----------+----------+----------+----------+------------------------------------------------|
| 0.348721 |   0.93071 | 0.923939 | 0.902529 | 0.978039 | xlm/cv-toxic-multi-en-finetune-unin

{'all': [0.9307099710583516,
  0.9327069447947064,
  0.933887187616337,
  0.9367279124785339,
  0.9334326476204202,
  0.9330017052755462],
 'es': [0.9254352754425739,
  0.9241968443955864,
  0.9213231369937371,
  0.918151795610982,
  0.9245697421417787,
  0.926174228774478],
 'it': [0.9177740931460417,
  0.9156800997294918,
  0.9188597920672685,
  0.9189636769546654,
  0.9134883323012744,
  0.9135993465436886],
 'tr': [0.9830083955223881,
  0.9832159514925374,
  0.9855993470149254,
  0.9848052705223881,
  0.9853381529850747,
  0.9871396921641792]}

In [32]:
# TODO change auc/all to all single model lb auc
for key in weights_dict:
  blend_byrank(weights_dict[key])
#   blend_byweight(weights_dict[key])
weights_dict

{'all': [0.047619047619047616,
  0.09523809523809523,
  0.23809523809523808,
  0.2857142857142857,
  0.19047619047619047,
  0.14285714285714285],
 'es': [0.23809523809523808,
  0.14285714285714285,
  0.09523809523809523,
  0.047619047619047616,
  0.19047619047619047,
  0.2857142857142857],
 'it': [0.19047619047619047,
  0.14285714285714285,
  0.23809523809523808,
  0.2857142857142857,
  0.047619047619047616,
  0.09523809523809523],
 'tr': [0.047619047619047616,
  0.09523809523809523,
  0.23809523809523808,
  0.14285714285714285,
  0.19047619047619047,
  0.2857142857142857]}

In [33]:
preds = []
for i in tqdm(range(len(dv))):
  lang = dfs[0].iloc[i].lang
  if lang not in weights_dict:
    lang = 'all'
  weights = weights_dict[lang]
  sum_weights = np.sum(weights)
  
  total = 0
  for j in range(len(dfs)):
    pred = dfs[j].iloc[i].pred
    pred *= weights[j]
    total += pred
   
  pred = total / sum_weights
  preds += [pred]
df = dfs[0].copy()
df.pred = preds

100%|██████████| 8000/8000 [00:16<00:00, 490.76it/s]


In [34]:
gezi.pprint(eval(df, 'blend_byrank'))


 +----------+-----------+----------+----------+----------+--------------+
|     loss |   auc/all |   auc/es |   auc/it |   auc/tr | name         |
|----------+-----------+----------+----------+----------+--------------|
| 0.194018 |  0.955195 | 0.932218 | 0.927377 |  0.98798 | blend_byrank |
+----------+-----------+----------+----------+----------+--------------+


In [35]:
# check test
dfs, dfs2 = [], []
for model in models:
  df, df2 = read_tests(f'{root}/{model}')
  dfs += [df]
  dfs2 += [df2]

In [36]:
preds = []
for i in tqdm(range(len(dt))):
  lang = dfs[0].iloc[i].lang
  if lang not in weights_dict:
    lang = 'all'
  weights = weights_dict[lang]
  sum_weights = np.sum(weights)
  
  total = 0
  for j in range(len(dfs)):
    pred = dfs[j].iloc[i].toxic
    pred *= weights[j]
    total += pred
   
  pred = total / sum_weights
  preds += [pred]
df = dfs[0].copy()
df.toxic = preds

100%|██████████| 63812/63812 [02:01<00:00, 524.10it/s]


In [37]:
df[df.toxic > 0.9].head(2)

,id,toxic,translated,lang,content
12,12,0.975860,"C is especially a sacred devotee and inner lining idiot, now quite disturbed, whose poems are pathetic and full of Jesus our Lord, Devil, God, sin, & Holy church of mother of god, a roman what! An idolater and celtomane to bottom. Poor Brittany! BLEI is the plural of BLAI, a wolf, or that which plunders, is the loot. Therefore its name from Aum to the nuts of the Bards dressed in toga roman & forgers are well known, BLEIMOR = PIRATE, PILLAGER. But c is a calottin, a true moron.",fr,"C est surtout un sacré dévot et calotin idiot, un mythomane assez dérangé dont les poèmes sont pathétiques et pleins de Jésus nôtre Seigneur, Diable, Dieu, péché & Sainte église de mère de dieu, un romain quoi! Un idolâtre et celtomane à fond. Pauvre Bretagne! BLEI est le pluriel de BLAI, un loup ou ce qui pille, fait du butin. Donc son nom de Secte à la noix des Bardes déguisés en toge romaine & faussaires notoires, BLEIMOR = PIRATE, PILLARD. Mais c est un calottin, un vrai crétin."
14,14,0.946066,"You want to offend them, go to edit in the center (which is a very simple and democratic, and, by the way), the fans of Inter that is the usa, not in color. I don't think there gremista was going to like it if I called the gaymista, bananas in pajamas, or talking about the shirt, it's a bar code of a bitch, you just move on to the market even once. Politics is not, cazzo!",pt,"Quer ofender, vai editar a desciclopédia (que é muito engraçada e democrática, por sinal) o torcedor do Inter é colorado, não colorido. Acho que nenhum gremista ia gostar se eu os chamasse de gaymista, de banana de pijama ou falar que a camisa é um código de barras de viado, basta passar no caixa do mercado que até faz bip. Politicagem não, cazzo!"


In [38]:
df[['id', 'toxic']].to_csv('../working/exps/submission.csv', index=False)

In [39]:
pd.read_csv('../working/exps/submission.csv').head()

,id,toxic
0,0,0.000210
1,1,0.000968
2,2,0.135567
3,3,0.000108
4,4,0.000115


In [40]:
df = pd.concat(dfs).groupby('id', as_index=False).mean()
df

,id,toxic
0,0,0.000204
1,1,0.000688
2,2,0.129575
3,3,0.000118
4,4,0.000131
...,...,...
63807,63807,0.085056
63808,63808,0.000122
63809,63809,0.019116
63810,63810,0.000324


In [41]:
df.to_csv('../working/exps/submission.csv', index=False)